In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.07217812538146973
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 16.743310129706813
200000 27.555874795750732
300000 30.513003806671588
400000 30.524493792696983
500000 31.067584432018908
600000 32.0146547349012
700000 30.950678969379055
800000 29.72448049629343
900000 30.074776802985355
1000000 30.318560031907445
1100000 13.538138133912108
1200000 27.125035683946525
1300000 30.011594788246004
1400000 29.921116985352594
1500000 30.444452557718687
1600000 30.6617892610277
1700000 32.03904368052476
1800000 30.13254529654953
1900000 30.570017949148212
2000000 31.74107230763663
2100000 31.130977382995063
2200000 8.542333616337938
2300000 34.418911927423196
2400000 36.9383513829495
2500000 36.97056601675082
2600000 36.252623835485466
2700000 36.62410940085019
2800000 35.006871414382005
2900000 34.65603524801906
3000000 36.20

29400000 27.51983521299297
29500000 29.80424685550589
29600000 29.661806937211388
29700000 29.531629593006592
29800000 28.72867815496308
29900000 16.81967214432159
30000000 30.122110423659986
30100000 31.540212487499904
30200000 28.25359897269743
30300000 19.471660476324075
30400000 22.27872217383936
30500000 29.41049288323085
30600000 29.529349047989896
30700000 27.982603149766298
30800000 29.841463866828686
30900000 26.52360822396934
31000000 26.571539984243675
31100000 24.23724921203354
31200000 12.863271767574226
31300000 26.838793271895476
31400000 25.92936077373072
31500000 27.162826813272424
31600000 29.014387184097952
31700000 27.489354121078467
31800000 30.917311198359286
31900000 28.074628040577522
32000000 12.966113893373308
32100000 32.26396763510763
32200000 29.128253935705022
32300000 32.08812162898291
32400000 29.49466995274657
32500000 12.276075535464306
32600000 33.18209740932918
32700000 29.34550623571927
32800000 31.422378537374524
32900000 28.190337945586197
3300000

59100000 10.494109469602474
59200000 26.16482165163707
59300000 25.58071483885358
59400000 25.94462699508861
59500000 25.421404089268968
59600000 27.891541954596384
59700000 12.061797890196605
59800000 29.004137265778837
59900000 29.414370795796135
60000000 29.096491906640516
60100000 29.670438298455355
60200000 28.079659835627595
60300000 27.96917770987348
60400000 27.960971234035654
60500000 27.16775804023905
60600000 15.391347470004174
60700000 22.209883046796868
60800000 25.305264387443646
60900000 25.247167488026257
61000000 24.636822024458187
61100000 24.691405329704782
61200000 12.79712267495355
61300000 21.827012364690827
61400000 24.950832797131234
61500000 24.212415427352145
61600000 24.42855101657581
61700000 25.941509388262258
61800000 15.093471599781585
61900000 24.622105417071293
62000000 27.469624357269872
62100000 28.48337312337791
62200000 28.695671286797385
62300000 27.790758168832657
62400000 28.85547575944582
62500000 27.51086252498612
62600000 27.861338305025185
62

88800000 15.122871269280934
88900000 22.036900871412897
89000000 21.72855784073915
89100000 12.572472562396687
89200000 18.979796132997482
89300000 21.880534579560436
89400000 22.400435008746122
89500000 11.474398793433508
89600000 20.720331916215844
89700000 22.496662025488178
89800000 22.64073382443156
89900000 23.01044395795496
90000000 24.873981618409875
90100000 24.0598023754345
90200000 25.521736052061982
90300000 23.306251051658204
90400000 24.608379367155237
90500000 24.264906056010418
90600000 23.065865633114893
90700000 23.05090259180333
90800000 20.909784827997424
90900000 10.919955320684844
91000000 21.142253002040786
91100000 22.29962944613939
91200000 21.722060080053275
91300000 9.245745225937865
91400000 21.86165101851382
91500000 22.128726100329143
91600000 16.30677494512599
91700000 17.405760398192207
91800000 21.491184258544184
91900000 22.185778087991796
92000000 22.292020875465237
92100000 23.79703605770172
92200000 25.737554890558506
92300000 23.707972355878542
924

117900000 20.552424574826727
118000000 24.06587691944827
118100000 21.937049562193987
118200000 23.199192322469536
118300000 23.120478099879637
118400000 20.20034047156142
118500000 22.090219602041874
118600000 20.479062238845394
118700000 20.75813212945935
118800000 21.633369078186284
118900000 20.79068974266903
119000000 13.849579257667896
119100000 17.33471395747268
119200000 6.813158871569951
119300000 14.236266549202792
119400000 15.856904974253073
119500000 21.53501479462623
119600000 20.791214984221995
119700000 21.558591039411997
119800000 22.52691306926579
119900000 20.675820929358302
120000000 22.444154694778874
120100000 22.13086345748075
120200000 23.511455766969746
120300000 23.741497367355922
120400000 22.173678276170538
120500000 22.02442710507052
120600000 20.943824778563986
120700000 20.949110915492565
120800000 21.99802273242033
120900000 19.80226060335237
121000000 21.31344167054933
121100000 19.86557232505071
121200000 8.653740420209697
121300000 13.208037569602649


...

289900000 5.8819796586173725
290000000 5.089670627664768
290100000 5.223081772613501
290200000 4.503492472453403
290300000 5.137483058752572
290400000 4.962508615959303
290500000 5.121417378553415
290600000 5.05445721353209
290700000 4.627778927290625
290800000 3.556081823148241
290900000 1.119589821486044
291000000 1.1212972136621104
291100000 5.81196839577573
291200000 4.615116915875608
291300000 5.084050014765157
291400000 4.991026318227956
291500000 4.909235722459936
291600000 4.788011158132189
291700000 4.701185205258857
291800000 5.209837404568745
291900000 4.905270264273904
292000000 6.003583707967278
292100000 5.281815340303127
292200000 5.153865353091169
292300000 5.1186076620991985
292400000 4.623739400951004
292500000 5.0643441801299485
292600000 4.871709014123567
292700000 5.121667281000157
292800000 4.642768509507645
292900000 5.162785310961247
293000000 1.1033624009438017
293100000 1.1053314792933013
293200000 3.9260137072031975
293300000 4.551305441251444
293400000 4.955

319000000 4.26195770062396
319100000 4.196966299496782
319200000 4.029696509396296
319300000 4.354094367461685
319400000 4.633695366575926
319500000 4.250942864110472
319600000 4.1140824147262105
319700000 4.311197217055137
319800000 4.416577112414984
319900000 5.560818853685384
320000000 5.60690851197844
320100000 4.84284845233457
320200000 4.583451472197222
320300000 4.368987225725646
320400000 4.71012217949125
320500000 4.8766451988006265
320600000 4.774973179408153
320700000 4.203670155710806
320800000 5.414313228263083
320900000 1.3176298244172553
321000000 0.8992079197443339
321100000 2.889048441040832
321200000 5.210515880560589
321300000 4.309199320123282
321400000 4.717179021651242
321500000 4.831704511773592
321600000 4.796435851837107
321700000 4.342369996788593
321800000 4.709857548273893
321900000 4.934509834921472
322000000 5.6213622870485755
322100000 6.624760279303355
322200000 5.481840388856972
322300000 5.506998417882249
322400000 5.040764004242492
322500000 5.2587104

347800000 1.6308491704293835
347900000 1.475100720827983
348000000 1.3893207054218026
348100000 1.3949248687579598
348200000 1.293775347367358
348300000 1.2352585873845419
348400000 1.2831389536764417
348500000 1.9146939245027066
348600000 1.5115986251405338
348700000 1.326890783535163
348800000 0.512388365348947
348900000 0.34184855910058026
349000000 0.8157408698126601
349100000 1.381511933520323
349200000 1.2927331235258566
349300000 1.497264451655741
349400000 1.3042518701156656
349500000 1.1853011344061122
349600000 1.3773868847953272
349700000 1.5862385136213957
349800000 1.3879506113672833
349900000 1.4680286935386964
350000000 1.5446472959581556
350100000 1.3257744813199575
350200000 1.3184743894547186
350300000 1.2603373155959288
350400000 1.1885782010205572
350500000 1.7819074056125237
350600000 1.6625768050515017
350700000 1.3018559273903496
350800000 1.2393843813639356
350900000 1.0174720926201761
351000000 0.3384388716426591
351100000 0.3405512316537658
351200000 1.3753706

375800000 0.18212182255219816
375900000 0.20452835943403444
376000000 0.17233059476690293
376100000 0.17387927472146747
376200000 0.17701722194299763
376300000 0.17709619530894358
376400000 0.17680697884248298
376500000 0.17546921907387245
376600000 0.1757106782822132
376700000 0.17458871418671013
376800000 0.1744463591997306
376900000 0.1723858600911518
377000000 0.17034868195917607
377100000 0.1699051147624638
377200000 0.17010936951845554
377300000 0.16956891570528748
377400000 0.21173232386134597
377500000 0.28533282097189755
377600000 0.17472054995230238
377700000 0.17187430154993866
377800000 0.17082230501975418
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 1.0237002783173204
378000000 1.120361598234364
378100000 0.9536241298085655
378200000 1.1234022921033144
378300000 0.9591295082060841
378400000 0.902824959370664
378500000 0.9573552104292432
378600000 1.2369713390621286
378700000 0.9316344716508282
378800000 0.9425058123105527
378900000 1.031

403700000 0.7846953101540586
403800000 0.776442583409755
403900000 0.6901436512947864
404000000 0.8504692515877424
404100000 0.6881985645905534
404200000 0.7837052540897389
404300000 0.7501177381396213
404400000 0.7996794824127489
404500000 0.7915278599705643
404600000 0.6830836180953204
404700000 0.8478110797297743
404800000 0.692393281743567
404900000 0.7470468620292128
405000000 0.7957461031223411
405100000 0.7946545046531551
405200000 0.758267553015111
405300000 0.6738086212026193
405400000 0.8204536205339868
405500000 0.6948258056828479
405600000 0.7043235721193419
405700000 0.8070057788456401
405800000 0.7510990385721147
405900000 0.7278144610509043
406000000 0.6433497175895055
406100000 0.7697488696678559
406200000 0.7113351401122378
406300000 0.6811999212346169
406400000 0.7368169199591259
406500000 0.747296002271084
406600000 0.6502231827618771
406700000 0.6093758289716721
406800000 0.6728438126013722
406900000 0.7181936468765887
407000000 0.637797189677614
407100000 0.6154054